# Reproduction of DECAF: Generating Fair Synthetic Data Using Causally-Aware Generative Networks

In this notebook we reproduce the results from the paper.

In [234]:
from xgboost import XGBClassifier
import numpy as np
import pandas as pd
import pytorch_lightning as pl
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, roc_auc_score
from sklearn.model_selection import train_test_split

## Loading the Adult dataset

In [235]:
def load_adult():
    """Load the Adult dataset in a pandas dataframe"""

    path = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
    names = [
        "age",
        "workclass",
        "fnlwgt",
        "education",
        "education-num",
        "marital-status",
        "occupation",
        "relationship",
        "race",
        "sex",
        "capital-gain",
        "capital-loss",
        "hours-per-week",
        "native-country",
        "income",
    ]
    df = pd.read_csv(path, names=names, index_col=False)
    df = df.applymap(lambda x: x.strip() if type(x) is str else x)

    for col in df:
        if df[col].dtype == "object":
            df = df[df[col] != "?"]
    
    return df

In [236]:
# Display examples from the dataset
adult_dataset = load_adult()
adult_dataset.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [237]:
def load_train_test_data(df):
    """Get GAN training data from Adult dataset"""
    replace = [
        [
            "Private",
            "Self-emp-not-inc",
            "Self-emp-inc",
            "Federal-gov",
            "Local-gov",
            "State-gov",
            "Without-pay",
            "Never-worked",
        ],
        [
            "Bachelors",
            "Some-college",
            "11th",
            "HS-grad",
            "Prof-school",
            "Assoc-acdm",
            "Assoc-voc",
            "9th",
            "7th-8th",
            "12th",
            "Masters",
            "1st-4th",
            "10th",
            "Doctorate",
            "5th-6th",
            "Preschool",
        ],
        [
            "Married-civ-spouse",
            "Divorced",
            "Never-married",
            "Separated",
            "Widowed",
            "Married-spouse-absent",
            "Married-AF-spouse",
        ],
        [
            "Tech-support",
            "Craft-repair",
            "Other-service",
            "Sales",
            "Exec-managerial",
            "Prof-specialty",
            "Handlers-cleaners",
            "Machine-op-inspct",
            "Adm-clerical",
            "Farming-fishing",
            "Transport-moving",
            "Priv-house-serv",
            "Protective-serv",
            "Armed-Forces",
        ],
        [
            "Wife",
            "Own-child",
            "Husband",
            "Not-in-family",
            "Other-relative",
            "Unmarried",
        ],
        ["White", "Asian-Pac-Islander", "Amer-Indian-Eskimo", "Other", "Black"],
        ["Female", "Male"],
        [
            "United-States",
            "Cambodia",
            "England",
            "Puerto-Rico",
            "Canada",
            "Germany",
            "Outlying-US(Guam-USVI-etc)",
            "India",
            "Japan",
            "Greece",
            "South",
            "China",
            "Cuba",
            "Iran",
            "Honduras",
            "Philippines",
            "Italy",
            "Poland",
            "Jamaica",
            "Vietnam",
            "Mexico",
            "Portugal",
            "Ireland",
            "France",
            "Dominican-Republic",
            "Laos",
            "Ecuador",
            "Taiwan",
            "Haiti",
            "Columbia",
            "Hungary",
            "Guatemala",
            "Nicaragua",
            "Scotland",
            "Thailand",
            "Yugoslavia",
            "El-Salvador",
            "Trinadad&Tobago",
            "Peru",
            "Hong",
            "Holand-Netherlands",
        ],
        [">50K", "<=50K"],
    ]

    for row in replace:
        df = df.replace(row, range(len(row)))

    df = df.values
    X = df[:, :].astype(np.uint32)
    X = StandardScaler().fit_transform(X)
    y = df[:, 14].astype(np.uint8)

    return train_test_split(X, y, test_size=2000, stratify=y)

In [238]:
# Define DAG for Adult dataset
dag = [
    # Edges from race
    ['race', 'occupation'],
    ['race', 'income'],
    ['race', 'hours-per-week'],
    ['race', 'education'],
    ['race', 'marital-status'],

    # Edges from age
    ['age', 'occupation'],
    ['age', 'hours-per-week'],
    ['age', 'income'],
    ['age', 'workclass'],
    ['age', 'marital-status'],
    ['age', 'education'],
    ['age', 'relationship'],
    
    # Edges from sex
    ['sex', 'occupation'],
    ['sex', 'marital-status'],
    ['sex', 'income'],
    ['sex', 'workclass'],
    ['sex', 'education'],
    ['sex', 'relationship'],
    
    # Edges from native country
    ['native-country', 'marital-status'],
    ['native-country', 'hours-per-week'],
    ['native-country', 'education'],
    ['native-country', 'workclass'],
    ['native-country', 'income'],
    ['native-country', 'relationship'],
    
    # Edges from marital status
    ['marital-status', 'occupation'],
    ['marital-status', 'hours-per-week'],
    ['marital-status', 'income'],
    ['marital-status', 'workclass'],
    ['marital-status', 'relationship'],
    ['marital-status', 'education'],
    
    # Edges from education
    ['education', 'occupation'],
    ['education', 'hours-per-week'],
    ['education', 'income'],
    ['education', 'workclass'],
    ['education', 'relationship'],
    
    # All remaining edges
    ['occupation', 'income'],
    ['hours-per-week', 'income'],
    ['workclass', 'income'],
    ['relationship', 'income'],
]

In [239]:
def dag_to_idx(df, dag):
    """Convert columns in a DAG to the corresponding indices."""

    dag_idx = []
    for edge in dag:
        dag_idx.append([df.columns.get_loc(edge[0]), df.columns.get_loc(edge[1])])

    return dag_idx

def create_bias_dict(df, edge_map):
    """
    Convert the given edge tuples to a bias dict used for generating
    debiased synthetic data.
    """
    bias_dict = {}
    for key, val in edge_map.items():
        bias_dict[df.columns.get_loc(key)] = [df.columns.get_loc(f) for f in val]
    
    return bias_dict

In [240]:
# Convert the DAG to one that can be provided to the DECAF model
dag_seed = dag_to_idx(adult_dataset, dag)
print(dag_seed)

[[8, 6], [8, 14], [8, 12], [8, 3], [8, 5], [0, 6], [0, 12], [0, 14], [0, 1], [0, 5], [0, 3], [0, 7], [9, 6], [9, 5], [9, 14], [9, 1], [9, 3], [9, 7], [13, 5], [13, 12], [13, 3], [13, 1], [13, 14], [13, 7], [5, 6], [5, 12], [5, 14], [5, 1], [5, 7], [5, 3], [3, 6], [3, 12], [3, 14], [3, 1], [3, 7], [6, 14], [12, 14], [1, 14], [7, 14]]


In [241]:
bias_dict_ftu = create_bias_dict(adult_dataset, {'income': ['sex']})
print('Bias dict FTU:', bias_dict_ftu)

bias_dict_dp = create_bias_dict(adult_dataset, {'income': [
    'occupation', 'hours-per-week', 'marital-status', 'education', 'sex',
    'workclass', 'relationship']})
print('Bias dict DP:', bias_dict_dp)

bias_dict_cf = create_bias_dict(adult_dataset, {'income': [
    'marital-status', 'sex']})
print('Bias dict CF:', bias_dict_cf)

Bias dict FTU: {14: [9]}
Bias dict DP: {14: [6, 12, 5, 3, 9, 1, 7]}
Bias dict CF: {14: [5, 9]}


In [242]:
# Get training and testing data
X_train, X_test, y_train, y_test = load_train_test_data(adult_dataset)

## Evaluate on original dataset

In [243]:
# Train an MLP
mlp = MLPClassifier().fit(X_train[:, :14], y_train)

/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [244]:
# Evaluate original data
y_pred = mlp.predict(X_test[:, :14])

print(
    precision_score(y_test, y_pred),
    recall_score(y_test, y_pred),
    roc_auc_score(y_test, y_pred),
)

0.8927875243664717 0.914780292942743 0.7917274958689617


## DECAF

### Train model

In [245]:
from models.DECAF import DECAF
from data import DataModule

dm = DataModule(X_train)

model = DECAF(
    dm.dims[0],
    dag_seed=dag_seed,
    h_dim=200,
    lr=0.5e-3,
    batch_size=64,
    lambda_privacy=0,
    lambda_gp=10,
    d_updates=10,
    alpha=2,
    rho=2,
    weight_decay=1e-2,
    grad_dag_loss=False,
    l1_g=0,
    l1_W=1e-4,
    p_gen=-1,
    use_mask=True,
)

trainer = pl.Trainer(max_epochs=10, logger=False)
trainer.fit(model, dm)

/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.9/site-packages/pytorch_lightning/core/datamodule.py:175: LightningDeprecationWarning: DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation("DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.")
/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.9/site-packages/pytorch_lightning/core/datamodule.py:170: LightningDeprecationWarning: DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation("DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.")
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:120: UserWarning: You passed in a `val_dataloader` but have no `validation_step`. Skipp

Initialised adjacency matrix as parsed:
 Parameter containing:
tensor([[0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 1.],
        [0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 1., 0., 1., 0.

In [246]:
def generate_synthetic_data(biased_edges={}):
    """Generate synthetic data which is also optionally debiased."""
    X_synth = (
        model.gen_synthetic(
            dm.dataset.x,
            gen_order=model.get_gen_order(),
            biased_edges=biased_edges,
        )
        .detach()
        .numpy()
    )

    X_synth = np.rint(X_synth)

    y_synth = X_synth[:,14]

    return X_synth[:, :14].astype(np.uint32), y_synth.astype(np.uint8)

In [247]:
def eval_model(mlp):
    """Helper function that prints evaluation metrics."""

    y_pred = mlp.predict(X_test[:, :14])

    print('Precision:', precision_score(y_test, y_pred))
    print('Recall:', recall_score(y_test, y_pred))
    print('AUROC:', roc_auc_score(y_test, y_pred))

### Evaluate DECAF-ND

In [248]:
# Generate synthetic data
X_synth, y_synth = generate_synthetic_data()

# Train downstream model
mlp = MLPClassifier().fit(X_synth, y_synth)

# Evaluate
eval_model(mlp)


Precision: 0.9175891758917589
Recall: 0.4966711051930759
AUROC: 0.6810664762913171


### Evaluate DECAF-FTU

In [249]:
# Generate synthetic data
X_synth, y_synth = generate_synthetic_data(biased_edges=bias_dict_ftu)

# Train downstream model
mlp = MLPClassifier().fit(X_synth, y_synth)

# Evaluate
eval_model(mlp)

Precision: 0.9563636363636364
Recall: 0.35019973368841545
AUROC: 0.6510034813020391


### Evaluate DECAF-DP

In [250]:
# Generate synthetic data
X_synth, y_synth = generate_synthetic_data(biased_edges=bias_dict_dp)

# Train downstream model
mlp = MLPClassifier().fit(X_synth, y_synth)

# Evaluate
eval_model(mlp)

Precision: 0.936
Recall: 0.31158455392809586
AUROC: 0.623663762907823


### Evaluate DECAF-CF

In [251]:
# Generate synthetic data
X_synth, y_synth = generate_synthetic_data(biased_edges=bias_dict_cf)

# Train downstream model
mlp = MLPClassifier().fit(X_synth, y_synth)

# Evaluate
eval_model(mlp)

Precision: 0.939209726443769
Recall: 0.41145139813581894
AUROC: 0.6655650564976283
